# Mini	projet	2	:	L’alignement et les PSSM

***

**PAQUET Michael (000410753) BA-INFO**


# Introduction

Maintenant que nous avons réussi à aligner une séquence avec une autre, et ce de façon global et local, il serait intéressant de pouvoir aligner une séquence à un profil. Mais qu'est-ce qu'un profil et à quoi peut-il servir ? 

Afin d'éclaircir cela, il est logique de revenir sur les domaines dont on a brièvement parlé précédemment lors du premier rapport d'analyse. Nous le savons, chaque protéine peut posséder des "domaines" représentant une sous-chaine de la protéine ayant des caractéristiques précises. Par exemple, le domaine WW dont il sera sujet ici est un domaine dans lequel sont conservés deux Tryptophanes. 

Il est évidemment pertinent d'essayer de voir si une protéine donnée possède de tels domaines, et c'est pour cela qu'on a créé les profils. Ceux-ci sont créés sur base de plusieurs séquences dont on sait qu'elles présentent un domaine. En alignant chacune des séquences les unes avec les autres, nous allons obtenir ce genre de résultat : 

<img src="exemple.png",width=400,height=300>

Sur cet exemple-ci, l'alignement met en évidence le domaine WW. L'implémentation d'un tel alignement ne sera pas implémenté ici. Nous utiliserons directement les résultats fournis par des outils tels que CLUSTAL.

Sur base de cela, nous pouvons créer un profil qui enregistrera d'une part les fréquences d’acides aminés à chaque position, et d'autre part l’importance évolutive de chaque acide aminé.
Profil qui sera ensuite aligné avec une séquence pour pouvoir déterminer si celle-ci possède ou non le domaine voulu.

Tout comme pour la partie précédente, nous allons donc procéder comme tel : 
- Implémentation des méthodes nécessaires
- Observation et discution sur les résultats obtenus
- Réponses aux problématiques


# Implémentation

Tout d'abord, réintroduisons le code déjà utilisé précedemment nécéssaire pour la suite :

In [1]:
import numpy as np
from math import sqrt
from math import log10

class Sequence:

    def __init__(self, sequence):
        self.sequence = sequence
        
    def __repr__(self):
        return "Sequence :{}".format(self.sequence)
    
    def __len__(self):
        return len(self.sequence)
    
    def __getitem__(self, index):
        return self.sequence[index]   


def parsingSequence(file):
    fichier = open(file, "r")
    reading = False
    sequence = ""
    allSequence = []
    
    for line in fichier:   #Pour chaques lignes dans le fichier
        
        if ">" == line[0] and not reading:  #Croisement du premier >
            reading = True 
            
        elif ">" == line[0] and reading or line[0] == "":   #On était en train de lire et on croise un ">"
            allSequence.append(Sequence(sequence)) 
            sequence = ""
            
        elif reading:     #On continue de lire la séquence
            sequence += line.replace("\n", "")
            
    allSequence.append(Sequence(sequence))
    return allSequence

In [2]:
class MatriceSub:

    def __init__(self, myFile):
        if isinstance(myFile, str):
            self.mat = self.parsingMat(myFile)
        else:
            self.mat = myFile
        self.column = ["A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","B","Z","X","*"]
        self.line = ["A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","B","Z","X","*"]
        self.acidesList = ["R","H","K","D","E","S","T","N","Q","C","G","P","A","I","L","M","F","W","Y","V","-"]

        
    def __repr__(self):
        return "Matrice de substitution :\n\n{}\n".format(self.mat)

    def __len__(self):
        return len(self.mat)

    def __getitem__(self, i):
        return self.mat[i]

    def index(self, elem):
        return self.acidesList.index(elem)

    def parsingMat(self, file):
        file = open(file, "r")
        reading = False
        mat = []
        
        for line in file:  #Pour chaque lignes dans le fichier
            
            if line[0:4] == "   A" or line[0:5] == "    A":   #On détecte le début de la matrice
                reading = True
                
            elif reading:
                
                if line[0] == "*":  #On détecte la fin
                    reading = False
                    
                ligne = line[2:].replace("\n", "").split(" ")
                voidNumber = 0
                
                for i in range(len(ligne)):
                    if ligne[i] == "": 
                        voidNumber += 1
                    else:
                        ligne[i] = int(ligne[i])
                        
                for j in range(voidNumber): #Suppression des caractères vides
                    ligne.remove("")
                    
                mat.append(ligne)
        return mat
    
    def findCoord(self, lineC, columnC):
        x = 0
        y = 0
        for i in range(len(self.line)):
            if self.line[i] == lineC:
                x = i
        for j in range(len(self.column)):
            if self.column[j] == columnC:
                y = j
        return (x, y)

    
    def readMat(self, lineC, columnC):
        coord = self.findCoord(lineC, columnC)
        return self.mat[coord[0]][coord[1]]

A noter ici que le code a quelque peu été modifié. Nous pouvons désormais créer une matrice de substitution à partir d'un profil.
En effet, lors de l'alignement de notre séquence avec le profil, nous n'utiliserons plus une matrice de substitution telle que Blosum ou Pam mais bien notre profil.

De plus amples informations sur ces deux ADT se trouvent dans le premier rapport d'analyse.

## Implémentation du Profil
> L'ADT que nous allons aborder ici représente donc un profil, ce dont nous avons parlé plus tôt lors de l'introduction

Cet ADT a pour but de construire la matrice PSSM, matrice dont nous ferons notre matrice de substitution lors de l'alignement.
Afin de créer ce profil, la liste des séquences est donnée au constructeur sur base de quoi la matrice PFM sera construite.

La matrice PFM représente le nombre de fois que l'acide aminé "i" a été croisé à la position "j" dans l'ensemble des séquences données. Pour cela rien de plus simple, il suffit de parcourir toutes les séquences et d'incrémenter les valeur dans la PFM en fonction de l'acide aminé rencontré ainsi que sa position dans la séquence. 

Une fois cette matrice obtenue, nous pouvons alors nous attaquer à la matrice qui nous intéresse le plus à savoir la PSSM. 
Il existe plusieurs façons de créer la PSSM. La manière ici choisie est celle faisant appel aux Pseudo-counts. Cela est efficace grâce aux actions qu'ont les Pseudo-counts sur la PSSM. 

Les pseudo-counts sont en fait des poids qu'on ajoute à la PSSM. Au plus le pseudo-count est élevé, au plus nous nous attendons à un résultat proche de la réalité. Il est donc important de savoir choisir la valeur attribuée à un tel élement. Cette valeur sera ici directement proportionnelle au nombre de séquences données (égale à la racine du nombre de séquences pour être précis) pour la création du profil. Ce choix s'avère tout à fait logique puisqu'au plus nous donnons de séquences à un profil, au plus ce profil est censé approcher un bon résultat, raison pour laquelle le pseudo-count sera plus élevé.

Le pseudo-count sera ici représenté par β.

Suite à un procédé mathématique, nous pouvons donc créer notre matrice PSSM représentant en fait la fréquence d'apparition d'un acide aminé "i" à la position "j". 


In [3]:
class Profile:

    def __init__(self, myFile, gap):
        self.pa = [0.0553, 0.0227, 0.0585, 0.0545, 0.0676, 0.0650, 0.0599, 0.0405, 0.0394, 0.0136, 0.0709, 0.0468, 0.0828, 0.0599, 0.0967, 0.0243, 0.0386, 0.0107, 0.0291, 0.0687, 0]
        self.acidesList = ["R","H","K","D","E","S","T","N","Q","C","G","P","A","I","L","M","F","W","Y","V","-"]
        self.probe = parsingSequence(myFile)
        self.β = sqrt(len(self.probe))
        self.gap = gap
        self.PFM = self.makePFM(self.probe)
        self.PSSM = self.makePSSM()


    def makePFM(self, probe):
        nbAcides = len(probe[0])
        nbSequences = len(probe)
        PFM = [[0 for i in range(nbAcides)] for j in range(len(self.acidesList))]

        for i in range (nbAcides):
            for j in range (nbSequences):
                acide = probe[j][i]
                PFM[self.acidesList.index(acide)][i] += 1

        return PFM


    def makePSSM(self):
        PSSM = [[0 for i in range(len(self.probe[0]))] for j in range(len(self.acidesList))]

        for i in range(len(self.acidesList)):
            p = self.pa[i]

            for j in range(len(self.probe[0])):
                n = self.PFM[i][j]
                if p != 0: #On a intégré le gap comme étant un acide aminé pour des facilités d'implémentations, et on lui a probabilité d'apparition "0" comme valeur sentinelle.
                    q = (n + self.β * p) / (len(self.probe) + self.β)
                    PSSM[i][j] = round(log10(q/p), 3)
                else:
                    PSSM[i][j] = self.gap

        return PSSM

- __--init--__
> Fonction d'initialisation : 
> - _self.pa_ représente les probabilités qu'un acide aminé soit présent n'importe où. Cela aurait pu être fait avec un dictionnaire.
> - _self.probe_ représente toutes les séquences qu'on donne au profil.

- __makePFM__
> Construit la matrice PFM dont on a déjà parlé. 

- __makePSSM__
> - Construit la matrice PSSM sur base de la PFM et du procédé mathématique qui suit la logique suivante :
> - $$PSSM[i][j] = m_{u,a} = \log{\frac{q_{u,a}}{p_{a}}}$$
> - $$q_{u,a} = \frac{n_{u,a} + βp_{a}}{N_{seq} + β}$$

#### Exemple d'execution

In [5]:
myFile = "test.fasta"
gap = -7
profile = Profile(myFile, gap)
acidesList = ["R","H","K","D","E","S","T","N","Q","C","G","P","A","I","L","M","F","W","Y","V","-"]
for i in range(len(profile.probe)):
    print(profile.probe[i])
print("\nProfile résultant de ces séquences :\n")
for i in range(len(profile.PSSM)):
    print(acidesList[i], end="")
    print(profile.PSSM[i])
    
print("\nL'acides aminé ", acidesList[0], " a une fréquence d'apparition à la position 2 de ", profile.PSSM[0][1])

Sequence :TGVEAENLLL
Sequence :PRAKAEESLS
Sequence :GRKDAERQLL

Profile résultant de ces séquences :

R[-0.436, 0.904, -0.436, -0.436, -0.436, -0.436, 0.622, -0.436, -0.436, -0.436]
H[-0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436]
K[-0.436, -0.436, 0.6, 0.6, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436]
D[-0.436, -0.436, -0.436, 0.628, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436]
E[-0.436, -0.436, -0.436, 0.543, -0.436, 0.989, 0.543, -0.436, -0.436, -0.436]
S[-0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, 0.558, -0.436, 0.558]
T[0.59, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436]
N[-0.436, -0.436, -0.436, -0.436, -0.436, -0.436, 0.747, -0.436, -0.436, -0.436]
Q[-0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, 0.758, -0.436, -0.436]
C[-0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436]
G[0.525, 0.525, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436, -0.436]
P[0.689, -0.436,

On voit en effet que l'Arginine apparait deux fois sur trois à la deuxième position. Il a donc une forte fréquence d'apparition. Tandis que ce même acide n'apparait nulle part d'autre, raison pour laquelle les autres valeurs pour cette même ligne sont toutes négatives. 

A noter que la fréquence d'apparition donc on parle n'est pas une probabilité. Les valeurs ne sont donc pas bornées entre 0 et 1.

## Implémentation de l'alignement d'une séquence à un profil

L'alignement d'une séquence avec un profil est en fait tout à fait similaire à l'alignement local entre deux séquences, méthode vue lors du premier rapport d'analyse.

La seule différence se situe sur le fait qu'au lieu de lire dans une matrice de substitution telle que Blosum ou Pam lors de la création de la matrice score et lors de la recherche de chemin, nous allons lire dans notre PSSM.

A noter que le code de la partie précédente n'est plus affiché ici afin d'éviter d'alourdir la cellule et d'augmenter la visibilité de ce qui est pertinent. Ce code aurait pu être réutilisé mais il a été choisi de faire de nouvelles méthodes pour les mêmes raisons.

In [6]:
class Alignement:

    def __init__(self, seq1, t, gap):
        self.seq1 = seq1
        self.t = t
        self.gap = gap
        self.score = []
        self.chemin = []
        self.fin = 0
        self.debut = 0
        self.result = ""

    def alignProfile(self, k):
        self.score = self.initPScore()
        for i in range(k):
            solution = []
            solution.append(np.max(self.score)) #Recherche du max
            i, j = np.where(self.score==solution[0]) #On prend l'indice de ce max
            i = i[0] #Il est possible qu'il y ai un max à plusieurs endroits
            j = j[0] #on choisi ici de prendre le premier
            align1 = ""
            self.fin = i
            self.debut, self.result = self.findProfilWays(int(i), int(j), align1, solution)
            print("Le domaine trouvé est : ", self.debut, "-", self.fin, "")
            print("Séquence résultante : ", self.result)


    def initPScore(self):
        score = []

        for i in range(len(self.seq1)):
            score.append([0 for j in range(len(self.t[0]))])
        s1 = 0
        for i in range(len(self.seq1)):
            s2 = 0
            for j in range(len(self.t[0])):
                score[i][j] = max(score[i-1][j-1] + self.t[self.t.index(self.seq1[s1])][s2], score[i-1][j] + self.t[self.t.index("-")][s2], score[i][j-1] + self.t[self.t.index("-")][s2-1], 0)
                score[i][j] = round(score[i][j], 3)
                s2 += 1
            s1 += 1

        return score


    def findProfilWays(self, i, j, align1, solution):
        self.chemin.append((i, j))
        if self.score[i][j] <= 0:
            self.reCalculate(self.chemin[-2][0], self.chemin[-2][1])
            return (i, align1[::-1])
        
        else:
            if i == 0:
                solution.append(self.score[i][j-1])
                self.score[i][j] = 0
                return self.findProfilWays(i, j-1, align1, solution)
                
            elif j == 0:
                solution.append(self.score[i-1][j])
                self.score[i][j] = 0
                return self.findProfilWays(i-1, j, align1, solution)
                
            else:
                if round(self.score[i-1][j-1] + self.t[self.t.index(self.seq1[i])][j], 3) == self.score[i][j]:
                    align1 += self.seq1[i]
                    solution.append(self.score[i-1][j-1])
                    self.score[i][j] = 0
                    return self.findProfilWays(i-1, j-1, align1, solution)

                elif round(self.score[i][j-1] + self.gap, 3) == self.score[i][j]:   
                    align1 += self.seq1[i]
                    solution.append(self.score[i][j-1])
                    self.score[i][j] = 0
                    return self.findProfilWays(i, j-1, align1, solution)

                elif round(self.score[i-1][j] + self.gap, 3) == self.score[i][j]:
                    align1 += "-"
                    solution.append(self.score[i-1][j])
                    self.score[i][j] = 0
                    return self.findProfilWays(i-1, j, align1, solution)  
                
    def reCalculate(self, k, m):
        #fonction de recalcul
        for i in range(k, len(self.seq1)):
            for j in range(m, len(self.t[0])):
                if (i, j) not in self.chemin: #On ne recalcul pas les valeurs si celles-ci étaient présentes dans le chemin trouvé
                    self.score[i][j] = max(self.score[i-1][j-1] + self.t[self.t.index(self.seq1[i])][j], self.score[i-1][j] + self.gap, self.score[i][j-1] + self.gap, 0)
                    self.score[i][j] = round(self.score[i][j], 3)

- __--init--__
> Fonction d'initialisation : 
> - self.debut et self.fin sont logiquement le début et la fin du domaine trouvé.
> - self.chemin permet de savoir par où on est passé lors du recalcul de la matrice score.
> - self.t est la PSSM.

- __initPScore__
> Va créer la matrice score sur base de la PSSM.
> - $$S(i,j) = \max{\left(S(i-1,j-1) + PSSM(seq(i),j), S(i-1, j) + gap, S(i, j-1) + gap, 0\right)}$$

- __findProfilWays__
> Cherche un chemin dans la matrice score. Il s'agit là de la même méthode utilisée pour l'alignement local lors du premier rapport d'analyse, sauf qu'au lieu de lire dans une matrice de substitution, nous lisons dans notre PSSM.

- __reCalculate__
> Recalcule la matrice Score à partir de l'endroit où nous nous étions arrêtés lors de la dernière recherche de chemin.

- __alignProfil__
> Cherche les k premiers alignements dans la matrice score. Cette fonction va donc initier la matrice score, et ensuite rechercher en celle-ci les chemins. Il s'agit là du même genre de méthode que dans le premier rapport d'analyse.

#### Exemple d'execution

In [7]:
myFile = "test.fasta"
gap = -7
profile = Profile(myFile, gap)
seq1 = "SRNAAEYLLS"
print("Alignement de la séquence", seq1, "avec le profil composé des séquences :\n", profile.probe, "\n")
mat = MatriceSub(profile.PSSM)
align = Alignement(seq1, mat, gap)
align.alignProfile(1)

Alignement de la séquence SRNAAEYLLS avec le profil composé des séquences :
 [Sequence :TGVEAENLLL, Sequence :PRAKAEESLS, Sequence :GRKDAERQLL] 

Le domaine trouvé est :  0 - 9 
Séquence résultante :  RNAAEYLLS


Cette exemple d'execution n'est pas très révélateur de l'efficacité de l'algorithme, mais une observation plus détaillée de résultats sera faite plus tard dans le rapport.

# Comparaison des résultats, réponse aux questions

Nous allons ici répondre aux demandes du projet, à savoir la résolution de questions et la comparaison/observation de nos résultats.

### Observation de la PSSM

Pour observer notre PSSM et pouvoir discuter de sa pertinence, il serait intéressant de la comparer avec un WebLogo de la famille WW. En construisant donc le WebLogo sur base de notre fichier contenant les séquences alignées, nous obtenons ceci :

<img src="weblogo.png",width=600,height=600>

Nous pouvons ici observer plusieurs choses :
- Comme dit plutôt, on voit qu'un Tryptophane est fortement voir entièrement conservé à la 6ème et 54ème place.
- Une Proline est fortement conservée à la 57ème place.
- Une Tyrosine et une Phénylalanine sont fortement conservées à la 18 et 19ème place.
- On peut ainsi observer beaucoup de choses.

Essayons donc d'afficher notre PSSM et trouver une correlation entre les deux !

In [8]:
myFile = "myresults-CLUSTAL.fasta"
gap = -7
profile = Profile(myFile, gap)
for i in range(len(profile.PSSM)-1):
    print(acidesList[i])
    print(profile.PSSM[i])

R
[-1.223, -0.704, -0.475, 0.038, -0.575, -1.223, -0.394, 0.216, 0.682, -0.325, -1.223, 0.09, -0.475, -0.266, -0.168, 0.89, 0.178, -1.223, -1.223, -1.223, -1.223, -0.214, -0.168, -0.575, -0.266, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -0.475, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, 0.694, 0.267, -0.475, -0.053, -1.223, -0.575, 0.569, -1.223, 0.751, -0.214]
H
[-1.223, -1.223, -0.403, -1.223, -0.403, -1.223, -1.223, -1.223, 0.629, 0.196, 0.24, -0.047, -0.136, 0.196, -1.223, -1.223, -0.25, -1.223, -1.223, -0.643, 0.533, 1.365, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -0.643, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -1.223, -0.403, -1.223, -1.223, -0.643, -1.223, -1.223, -0.643, -1.223, 0.35, 0.785, -1.223, 0.028, -1.223]
K
[-0.593, -1.223, -0.043, -0.347, -0.593, -1.223, -0.043, 0.404, 0.015, 0

L'affichage est quelque peu désordonné, mais il reste facile d'observer les résultats.

- Allons à la ligne "W" et comparons cela : dans notre PSSM, on observe deux valeurs hautes à savoir 1.937 et 1.853 à la 6ème place et à la 54ème place, tout comme le montre le Weblogo !
- Pour la Proline maintenant, on observe une valeur haute à savoir 1.294 à la 57ème, une fois de plus comme le montre le Weblogo. On peut également vérifier qu'à la 57ème place, aucun autre acide aminé n'a une valeur positive puisqu'aucun d'entre eux n'est bien conservé à cette place là. Petite observation supplémentaire : Malgré le fait que sur le Weblogo, la Proline prend une place aussi importante que les Tryptophanes aux places n°6 et n°54, celle-ci n'a pas forcément une valeur égale aux Tryptophanes dans la PSSM.
- On observe que tout comme le Weblogo le montre, les Tyrosines et Phénylalanines sont les seuls acides aminés à avoir une valeur haute pour la 18ème et 19ème place.

On pourrait ainsi observer l'ensemble des résultats, mais il semble clair qu'il existe une forte corrélation entre les deux structures. Cela est bien évidemment logique puisque ces deux objets sont tout deux censés nous montrer quels sont les acides aminés bien conservés pour chaque position dans la séquence.

Continuons maintenant notre comparaison avec une nouvelle structure : les HMM-logo. Pour cela, on se rend sur le site PFAM et on affiche le HMM-logo pour la famille WW. Voici donc ce HMM-logo :


<img src="hmmlogo.png",width=600,height=600>

Il n'est nul besoin d'à nouveau observer la totalité du logo ici-présent, puisque celui-ci ressemble fortement au Weblogo déjà observé.
A noter tout de même que celui-ci explique peut-être une de nos observations quant aux Prolines : On voit ici que contrairement au Weblogo, la Proline n'occupe pas une place aussi importante que dans le Weblogo, ce qui induit le fait que la valeur observée dans notre PSSM pour la Proline à cette place là n'est pas aussi haute que celles des Tryptophanes aux 6ème et 54ème place.

Il est cependant important de se rendre compte du fait que le nom de "WW domain" parrait maintenant tout à fait logique. Il s'agit là, comme dit précédemment, d'un domaine dans lequel deux Tryptophanes sont fortement conservés à deux endroits distincts dans la séquence.

Petit complément d'information sur les deux Logos importés. On peut expliquer leur légère différence de la façon suivante : Puisque notre WebLogo a été créé sur base de notre fichier "myresult-CLUSTAL" qui contient nos 136 domaines WW, il est supposé logique qu'il diffère du HMM-logo qui lui se base sur l'entièreté des domaines WW. On peut donc suggérer que le HMM-logo ici-présent est plus proche de la réalité que notre Web-logo. 

### Recherche des domaines WW dans nos protéines

Commençons d'abord par rechercher les domaines WW des protéines présentes dans le fichier protéine-séquence.fasta.
Pour cela, nous devons tout d'abord créer notre profil et donc chercher les séquences nécessaires à cette création.
Il nous est pour cela conseillé de nous rendre sur le site SMART sur lequel est disponnible un ensemble de séquences qui représente la famille WW.

Une fois ces séquences collectées, il nous faut les aligner, et ce via CLUSTAL, TCoffee ou MUSCLE. Nous choisirons ici CLUSTAL.

Maintenant que nous avons nos séquences alignées présentes dans un fichier, nous pouvons créer un profil adéquat et répondre à la problématique. 

Commençons d'abord par afficher les domaines WW repris d'Uniprot, et affichons ensuite nos résultats pour pouvoir les comparer.
Voici les domaines d'Uniprot: 

<img src="domaine.png",width=600,height=800>

Et voici maintenant nos résultats : 

In [9]:
myFile = "myresults-CLUSTAL.fasta"
gap = -4
profile = Profile(myFile, gap)
mat = MatriceSub(profile.PSSM)
sequences = parsingSequence("protein-sequences.fasta")

for i in range (len(sequences)):
    seq1 = sequences[i]
    print("Alignement de la séquence numéro :", i+1)
    align = Alignement(seq1, mat, gap)
    align.alignProfile(1)
    print()

Alignement de la séquence numéro : 1
Le domaine trouvé est :  228 - 252 
Séquence résultante :  LPDGWEQAMTQQDGEIYYINHKN

Alignement de la séquence numéro : 2
Le domaine trouvé est :  609 - 632 
Séquence résultante :  LPPGWEERQDIILGRTYYVNHES

Alignement de la séquence numéro : 3
Le domaine trouvé est :  144 - 162 
Séquence résultante :  WTEHKSPPDGRTYYYNTET

Alignement de la séquence numéro : 4
Le domaine trouvé est :  5 - 28 
Séquence résultante :  LPPGWEKRMSRSSGRVYYFNHIT



Nous trouvons donc un domaine pour chacune des protéines. Comparons ces résultats avec Uniprot. 

Pour ce qui est de YAP1_HUMAN : 
- Notre résultat : 228-252
- Uniprot :        230-263

NEDD4_HUMAN :
- Notre résultat : 609-632
- Uniprot :        610-643

PR40A_HUMAN :
- Notre résultat : 144-162
- Uniprot :        140-173

PIN1_HUMAN :
- Notre résultat : 5-28
- Uniprot :        5-39

Nous pouvons observer que pour chacune des protéines nos résultats sont très proches de la vérité. Ils sont certes précis, mais pas à 100%. En effet, notre algorithme ne trouve à chaque fois qu'une partie seulement du domaine et non le domaine entier. Cela peut s'expliquer par le fait que dans notre profil, après le 24ème acide aminé, une grosse séquence de gap apparait : 

<img src="gap.png",width=400,height=300>

Il est donc normal que notre alignement n'aligne pas les séquences avec cette partie du profil et que nous nous retronvons donc avec des domaines dont la taille est de l'ordre de ~25 acides aminés.

On peut si on le souhaite réitérer l'opération avec un gap différent et observer de nouveaux résultats.



In [10]:
gap = -0.5
profile = Profile(myFile, gap)
mat = MatriceSub(profile.PSSM)

for i in range (len(sequences)):
    seq1 = sequences[i]
    print("Alignement de la séquence numéro :", i+1)
    align = Alignement(seq1, mat, gap)
    align.alignProfile(1)
    print()

Alignement de la séquence numéro : 1
Le domaine trouvé est :  160 - 192 
Séquence résultante :  LPAGWEMAKTSSSGQRYFLNHI

Alignement de la séquence numéro : 2
Le domaine trouvé est :  864 - 913 
Séquence résultante :  LPPGWEERTHTTDGRIFYINHN

Alignement de la séquence numéro : 3
Le domaine trouvé est :  167 - 203 
Séquence résultante :  L-KCPWKEYKSDSSGKPYYYNSQT

Alignement de la séquence numéro : 4
Le domaine trouvé est :  5 - 28 
Séquence résultante :  LPPGWEKRMSRSSGRVYYFNHIT



Cette fois ci, notre résultat pour YAP1_HUMAN trouve un nouveau domaine. Il s'agit là en fait d'un autre domaine WW de la protéine. En effet celle-ci en possède deux nous indique Uniprot, à savoir 230-262 comme dit précédemment, mais aussi 171-204.

Tandis qu'il se passe quelque chose d'étrange à première vue pour PR40A_HUMAN et NEDD4_HUMAN. En effet, l'intervale trouvé semble entre les deux réels domaines des protéines.

PR40A_HUMAN :
- Interval trouvé : 167-203
- Uniprot : WW1 = 140-173, WW2 = 181-214

NEDD4_HUMAN
- Interval trouvé : 864-913
- Uniprot : WW3 = 	840 – 873, WW4 = 892 – 925

On pourrait expliquer cela par le fait que puisque deux domaines WW sont proches l'un de l'autre dans la protéine, le deuxième Tryptophan du premier domaine est proche du premier Tryptophan du deuxième domaine. Notre algorithme, pas aussi précis qu'on le souhaitrait, pense peut-être qu'il s'agit donc là d'un domaine WW.

Il semble en tout cas une mauvaise idée de prendre -0.5 pour gap puisque l'efficacité de l'algorithme s'en retrouve réduit.

Qu'en est-il maintenant lorsque nous essayons de faire un alignement avec une valeur pour k > 1 ? 

In [11]:
myFile = "myresults-CLUSTAL.fasta"
gap = -4
profile = Profile(myFile, gap)
mat = MatriceSub(profile.PSSM)
sequences = parsingSequence("protein-sequences.fasta")

for i in range (len(sequences)):
        seq1 = sequences[i]
        print("Alignement de la séquence numéro :", i+1)
        align = Alignement(seq1, mat, gap)
        align.alignProfile(3)
        print()

Alignement de la séquence numéro : 1
Le domaine trouvé est :  228 - 252 
Séquence résultante :  LPDGWEQAMTQQDGEIYYINHKN
Le domaine trouvé est :  166 - 192 
Séquence résultante :  LPAGWEMAKTSSSGQRYFLNHI
Le domaine trouvé est :  261 - 263 
Séquence résultante :  

Alignement de la séquence numéro : 2
Le domaine trouvé est :  609 - 632 
Séquence résultante :  LPPGWEERQDIILGRTYYVNHES
Le domaine trouvé est :  888 - 913 
Séquence résultante :  LPPGWEERTHTTDGRIFYINHN
Le domaine trouvé est :  764 - 789 
Séquence résultante :  LPPGWEEKQDEERGRSYYVDHNS

Alignement de la séquence numéro : 3
Le domaine trouvé est :  144 - 162 
Séquence résultante :  WTEHKSPPDGRTYYYNTET
Le domaine trouvé est :  180 - 203 
Séquence résultante :  SKCPWKEYKSDSSGKPYYYNSQT
Le domaine trouvé est :  162 - 172 
Séquence résultante :  KQSTWEKPDD

Alignement de la séquence numéro : 4
Le domaine trouvé est :  5 - 28 
Séquence résultante :  LPPGWEKRMSRSSGRVYYFNHIT
Le domaine trouvé est :  30 - 36 
Séquence résultante :  SQWERP


Aligner avec une valeur pour k>1 s'avère être pertinent ! En effet, on peut observer deux choses intéressantes : 
- Pour la première protéine, par exemple, qui comporte deux domaines WW, produire un alignement avec une valeur pour k>1 permet de trouver ces deux domaines ! On observe la même chose pour les autres protéines comportant plusieurs domaines WW.
- Pour la troisième protéine maintenant, on observe qu'à la troisième itération, notre algorithme trouve un nouveau bout de domaine non trouvé précédemment. Le domaine "140-173" dont on avait trouvé la partie "144-162" est maintenant complètée par la partie "162-172". On peut donc supposer selon nos résultats que le domaine est "144-172", ce qui représente un résultat encore plus proche de la réalité que ce qu'il était avant ! 

Le genre de résultat dont on parle au deuxième point peut à nouveau facilement s'expliquer. Prenons à nouveau cette image-ci :

<img src="partie.png",width=400,height=300>

A la première itération, notre algorithme aligne notre protéine avec la première partie indiquée sur le schéma. Celui-ci s'arrête après pour les raisons déjà expliquées plus haut. Ensuite, lors d'une nouvelle itération, notre algorithme aligne simplement notre séquence avec la deuxième partie du profil mis en évidence dans le schéma. Deuxième partie qui par ailleurs a une taille de 10 acides aminés, tout comme le montre notre résultat (162-172).
Ceci est bien évidemment un cas général et non un résultat présent dans 100% des cas. On peut par exemple observer pour la dernière protéine qu'à une deuxième itération sur le même domaine, on ne trouve pas "28-39" mais "30-36". Notre algorithme n'est donc pas aussi précis qu'on le souhaitrait, comme déjà dis plus haut.

Cette non-précision permet d'ouvrir une légère parenthèse : On pourrait améliorer notre algorithme en implémentant un gap affine, mais ceci n'a pas été fait lors de ce projet-ci. Diverses informations quant à ce genre d'implémentation se trouve dans le pdf nommé "RM profiles and alignements" joint avec le rapport. Les gaps affines ne représentent pas la seule amélioration possible. En effet, la construction de notre PSSM se fait via une méthode utilisant les pseudo-counts, mais il existe une méthode améliorée utilisant également les pseudo-counts.

# Conclusion

Après avoir réussi à aligner deux séquences entre elles, nous avons maintenant réussi à aligner une séquence à un profil afin de trouver un domaine voulu au sein de celle-ci, ce qui répond pleinement aux attentes de la problématique. En plus de cela, après comparaison avec Uniprot, nous avons pu prouver la pertinence de nos résultats. 

Des améliorations restent cependant possibles afin de rendre plus précis notre algorithme. 